In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os


# Nominal scenario - Comparison between controllers

In [ ]:
# Load summary stats
summary_pid = pd.read_csv("simulation_results/statistics/results_pid_nominal_aggregated_low_var.csv")
summary_onoff = pd.read_csv("simulation_results/statistics/results_onoff_nominal_aggregated_low_var.csv")
summary_fuzzy = pd.read_csv("simulation_results/statistics/results_fuzzy_nominal_aggregated_low_var.csv")

summary_pid['source'] = 'PID'
summary_onoff['source'] = 'ON/OFF'
summary_fuzzy['source'] = 'Fuzzy'

summary_stats = pd.concat([summary_pid, summary_onoff, summary_fuzzy])

# Load per-run results
results_pid = pd.read_csv("simulation_results/statistics/results_pid_nominal_all_low_var.csv" )
results_onoff = pd.read_csv("simulation_results/statistics/results_onoff_nominal_all_low_var.csv")
results_fuzzy = pd.read_csv("simulation_results/statistics/results_fuzzy_nominal_all_low_var.csv")

# Label sources (if not consistent already)
results_pid['controller'] = 'PID'
results_onoff['controller'] = 'ON/OFF'
results_fuzzy['controller'] = 'Fuzzy'

results = pd.concat([results_pid, results_onoff, results_fuzzy])

palette = {'PID': 'skyblue', 'ON/OFF': 'salmon', 'Fuzzy': 'lightgreen'}

In [ ]:

def boxplot_of_metrics(results, summary_stats, palette):
    """
    Draws boxplots for each metric in the results DataFrame,
    overlaying mean and confidence intervals from summary_stats.
    """
    metrics = results['metric'].unique()

    for metric in metrics:
        plt.figure(figsize=(10, 6))

        # Filter data
        metric_results = results[results['metric'] == metric]
        metric_results_clean = metric_results.dropna(subset=['value'])
        print(metric_results_clean.head())
        metric_summary = summary_stats[summary_stats['metric'] == metric]

        # Draw boxplot
        ax = sns.boxplot(
            data=metric_results_clean,
            x='controller', y='value',
            palette=palette,
            hue='controller',
            showmeans=False
        )

        # Add mean + CI overlay
        for i, row in metric_summary.iterrows():
            x_pos = ['PID', 'ON/OFF', 'Fuzzy'].index(row['source'])

            # Mean as black dot
            ax.plot(x_pos, row['mean'], 'o', color='black')

            # CI for mean
            ax.errorbar(x=x_pos, y=row['mean'],
                       yerr=[[row['mean'] - row['mean_ci_lower']], [row['mean_ci_upper'] - row['mean']]],
                        fmt='none', ecolor='black', capsize=5, lw=2)

        plt.title(f"{metric} — Boxplot with Mean and CI")
        plt.xlabel("Controller")
        plt.ylabel(metric)
        plt.show()


In [ ]:
boxplot_of_metrics(results, summary_stats, palette)

# Noise scenario - Comparison between controllers

In [ ]:
# Load summary stats
summary_pid = pd.read_csv("simulation_results/statistics/results_pid_with_noise_aggregated_low_var.csv")
summary_onoff = pd.read_csv("simulation_results/statistics/results_onoff_with_noise_aggregated_low_var.csv")
summary_fuzzy = pd.read_csv("simulation_results/statistics/results_fuzzy_with_noise_aggregated_low_var.csv")

summary_pid['source'] = 'PID'
summary_onoff['source'] = 'ON/OFF'
summary_fuzzy['source'] = 'Fuzzy'

summary_stats_noise = pd.concat([summary_pid, summary_onoff, summary_fuzzy])

# Load per-run results
results_pid = pd.read_csv("simulation_results/statistics/results_pid_with_noise_all_low_var.csv" )
results_onoff = pd.read_csv("simulation_results/statistics/results_onoff_with_noise_all_low_var.csv")
results_fuzzy = pd.read_csv("simulation_results/statistics/results_fuzzy_with_noise_all_low_var.csv")

# Label sources (if not consistent already)
results_pid['controller'] = 'PID'
results_onoff['controller'] = 'ON/OFF'
results_fuzzy['controller'] = 'Fuzzy'

results_noise = pd.concat([results_pid, results_onoff, results_fuzzy])


In [ ]:
boxplot_of_metrics(results_noise, summary_stats_noise, palette)

# Disturbances scenario 

In [ ]:
# Load summary stats
summary_pid = pd.read_csv("simulation_results/statistics/results_pid_with_disturbances_aggregated.csv")
summary_onoff = pd.read_csv("simulation_results/statistics/results_onoff_with_disturbances_aggregated.csv")
summary_fuzzy = pd.read_csv("simulation_results/statistics/results_fuzzy_with_disturbances_aggregated.csv")

summary_pid['source'] = 'PID'
summary_onoff['source'] = 'ON/OFF'
summary_fuzzy['source'] = 'Fuzzy'

summary_stats_disturbances_ = pd.concat([summary_pid, summary_onoff, summary_fuzzy])

# Load per-run results
results_pid = pd.read_csv("simulation_results/statistics/results_pid_with_disturbances_all_low_var.csv" )
results_onoff = pd.read_csv("simulation_results/statistics/results_onoff_with_disturbances_all_low_var.csv")
results_fuzzy = pd.read_csv("simulation_results/statistics/results_fuzzy_with_disturbances_all_low_var.csv")

# Label sources (if not consistent already)
results_pid['controller'] = 'PID'
results_onoff['controller'] = 'ON/OFF'
results_fuzzy['controller'] = 'Fuzzy'

results_disturbances_ = pd.concat([results_pid, results_onoff, results_fuzzy])


In [ ]:

boxplot_of_metrics(results_disturbances_, summary_stats_disturbances_, palette)